In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 8, Finished, Available, Finished)

## **READING SHORTCUT**


In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/RawData-DynamicAdventureWorks_Customers/AdventureWorks_Customers.csv")
# df now is a Spark DataFrame containing CSV data from "Files/RawData-DynamicAdventureWorks_Customers/AdventureWorks_Customers.csv".
display(df)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1e1366f3-788a-4116-acc8-3984c6102ba6)

## **READING FROM BRONZE LAKEHOUSE**

**Reading Customer Data & Apllying Transformations**

In [ ]:
df1 = spark.read.format("csv")\
            .option("header","True")\
            .option("inferSchema","True")\
            .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Files/RawData-DynamicAdventureWorks_Customers/AdventureWorks_Customers.csv")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 10, Finished, Available, Finished)

In [ ]:
display(df1)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9bcf76d8-2b78-4e26-83cb-9b4785d42702)

In [ ]:
df1.printSchema()

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 12, Finished, Available, Finished)

root
 |-- CustomerKey: integer (nullable = true)
 |-- Prefix: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- BirthDate: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- AnnualIncome: string (nullable = true)
 |-- TotalChildren: integer (nullable = true)
 |-- EducationLevel: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- HomeOwner: string (nullable = true)



In [ ]:
df.printSchema()

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 13, Finished, Available, Finished)

root
 |-- CustomerKey: string (nullable = true)
 |-- Prefix: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- BirthDate: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- AnnualIncome: string (nullable = true)
 |-- TotalChildren: string (nullable = true)
 |-- EducationLevel: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- HomeOwner: string (nullable = true)



In [ ]:
df1 = df1.withColumn("FullName",concat(col("Prefix"),lit(" "),col("FirstName"),lit(" "),col("LastName")))
display(df1)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 85286c4d-bff0-4246-b446-d1c56e2d7294)

In [ ]:
df1 = df1.withColumn("Domain",split("EmailAddress","@")[1])
display(df1)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 925da120-3a3d-4130-963b-36b001c7b683)

In [ ]:
df1.write.format("delta")\
            .mode("append")\
            .option("path","Files/customers")\
            .saveAsTable("customers")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 16, Finished, Available, Finished)

In [ ]:
df1.write.format("delta")\
            .mode("append")\
            .option("path","Tables/customers")\
            .saveAsTable("customers1")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 17, Finished, Available, Finished)

**Reading Customers data and applying transformations**

In [ ]:
dff1 = spark.read.format("csv")\
            .option("header","true")\
             .option("inferSchema","True")\
            .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Files/RawData_Dynamic_Adventure_Works_Sales_2016/AdventureWorks_Sales_2016.csv")

dff2 = spark.read.format("csv")\
            .option("header","true")\
             .option("inferSchema","True")\
            .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Files/RawData-DynamicAdventureWorks_Sales_2017/AdventureWorks_Sales_2017.csv")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 23, Finished, Available, Finished)

In [ ]:
df2 = dff1.union(dff2)
display(df2)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 23d9daf8-c879-43ae-b228-7effbfa21995)

In [ ]:
df2.write.format("delta")\
            .mode("append")\
            .option("path","Files/sales")\
            .saveAsTable("sales")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 25, Finished, Available, Finished)

**Reading Cat and Subcat Data and applying transformations and write data at the end**

In [ ]:
df3 = spark.read.format("delta")\
        .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Tables/Dataflow")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 26, Finished, Available, Finished)

In [ ]:
display(df3)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3c1c858-df4b-4132-9984-cf9d47c2720f)

In [ ]:
df3.write.format("delta")\
            .mode("append")\
            .option("path","Files/subcat")\
            .saveAsTable("subcat")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 28, Finished, Available, Finished)

**Reading Calender Data and applying transformations and write data at the end**

In [ ]:
df4 = spark.read.format("csv")\
            .option("header","true")\
             .option("inferSchema","True")\
            .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Files/RawData-DynamicAdventureWorks_Calendar/AdventureWorks_Calendar.csv")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 29, Finished, Available, Finished)

In [ ]:
display(df4)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 31eb717a-a480-4929-a221-4e028f33d478)

In [ ]:
df4.write.format("delta")\
            .mode("append")\
            .option("path","Files/calender")\
            .saveAsTable("calender")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 31, Finished, Available, Finished)

**Reading Products Data and applying transformations and write data at the end**

In [ ]:
df5 = spark.read.format("csv")\
            .option("header","true")\
             .option("inferSchema","True")\
            .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Files/RawData-DynamicAdventureWorks_Products/AdventureWorks_Products.csv")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 32, Finished, Available, Finished)

In [ ]:
display(df5)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e6a4ca28-9dfa-49a8-a733-fe97899fffed)

In [ ]:
df5 = df5.withColumn("ProductsSKU",split("ProductSKU","-")[0])
display(df5)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2aaad3e7-30c1-4940-9e04-5a7032143b74)

In [ ]:
df5.write.format("delta")\
            .mode("append")\
            .option("path","Files/products")\
            .saveAsTable("products")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 35, Finished, Available, Finished)

**Reading Returns Data and applying transformations and write data at the end**

In [ ]:
df6 = spark.read.format("csv")\
            .option("header","true")\
             .option("inferSchema","True")\
            .load("abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/c7ce1fe0-b5f4-4568-b02b-6c6bf105f878/Files/Adls/Returns.csv")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 36, Finished, Available, Finished)

In [ ]:
display(df6)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4577f18b-af3f-46e5-9f06-27fdf91fe46b)

In [ ]:
df6.write.format("csv")\
            .mode("append")\
            .option("path","Files/returns")\
            .saveAsTable("returns")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 38, Finished, Available, Finished)

**SPARK UTILITIES**

In [ ]:
from notebookutils import mssparkutils

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 39, Finished, Available, Finished)

In [ ]:
mssparkutils.fs.help()

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 40, Finished, Available, Finished)

In [ ]:
mssparkutils.fs.ls("Files")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 41, Finished, Available, Finished)

[FileInfo(path=abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/3f9a2f1a-26cd-4544-afcc-d5155171c6cf/Files/RawData-DynamicAdventureWorks_Customers, name=RawData-DynamicAdventureWorks_Customers, size=0),
 FileInfo(path=abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/3f9a2f1a-26cd-4544-afcc-d5155171c6cf/Files/calender, name=calender, size=0),
 FileInfo(path=abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/3f9a2f1a-26cd-4544-afcc-d5155171c6cf/Files/customers, name=customers, size=0),
 FileInfo(path=abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/3f9a2f1a-26cd-4544-afcc-d5155171c6cf/Files/products, name=products, size=0),
 FileInfo(path=abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft.com/3f9a2f1a-26cd-4544-afcc-d5155171c6cf/Files/returns, name=returns, size=0),
 FileInfo(path=abfss://d4759ed9-e0be-4822-90e8-1e0f8e67de08@onelake.dfs.fabric.microsoft

In [ ]:
mssparkutils.fs.mkdirs("Files/mkdir")

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 43, Finished, Available, Finished)

True

In [ ]:
mssparkutils.fs.cp("Files/calender","Files/mkdir",True)

StatementMeta(, c5de1ff6-823b-4975-8005-2e4a32a26fce, 45, Finished, Available, Finished)

True